<a href="https://colab.research.google.com/github/SasinduShanaka/BMW_price_predict_model/blob/main/BMW_carPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import csv
import pandas as pd
df = pd.read_csv("bmw.csv")


In [19]:
df.head()
df.shape
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 10079 entries, 0 to 10783
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         10079 non-null  object 
 1   year          10079 non-null  int64  
 2   price         10079 non-null  float64
 3   transmission  10079 non-null  object 
 4   mileage       10079 non-null  float64
 5   fuelType      10079 non-null  object 
 6   tax           10079 non-null  float64
 7   mpg           10079 non-null  float64
 8   engineSize    10079 non-null  float64
dtypes: float64(5), int64(1), object(3)
memory usage: 787.4+ KB


,year,price,mileage,tax,mpg,engineSize
count,10079.000000,10079.000000,10079.000000,10079.000000,10079.000000,10079.000000
mean,2016.944737,20788.950888,26912.336740,130.804643,57.797748,2.333406
std,2.359491,8178.790984,25321.767352,62.992394,37.168690,15.958635
min,1996.000000,1200.000000,1.000000,0.000000,5.500000,0.000000
25%,2016.000000,14498.000000,6270.500000,125.000000,47.900000,2.000000
50%,2017.000000,19795.000000,20685.000000,145.000000,54.300000,2.000000
75%,2019.000000,26430.000000,40000.000000,145.000000,62.800000,2.000000
max,2020.000000,44500.000000,214000.000000,580.000000,1487.000000,1500.000000


Data Quality Report

In [18]:
df.isnull().sum()

,0
model,0
year,0
price,0
transmission,0
mileage,0
fuelType,0
tax,0
mpg,0
engineSize,0


In [6]:
#How to handle


#Numerical columns → fill with median

df['mileage'].fillna(df['mileage'].median(), inplace=True)
df['engineSize'].fillna(df['engineSize'].median(), inplace=True)
df['tax'].fillna(df['tax'].median(), inplace=True)
df['mpg'].fillna(df['mpg'].median(), inplace=True)
df['price'].fillna(df['price'].median(), inplace=True)

#Categorical columns → fill with mode

df['transmission'].fillna(df['transmission'].mode()[0], inplace=True)
df['fuelType'].fillna(df['fuelType'].mode()[0], inplace=True)


/tmp/ipython-input-522456203.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['mileage'].fillna(df['mileage'].median(), inplace=True)
/tmp/ipython-input-522456203.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

In [11]:
#handelig duplicate values

df.duplicated().sum()  #this is to check the duplicate
df.drop_duplicates(inplace=True) #thus is removing duplicates


utlier Detection

In [13]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

df = df[(df['price'] >= Q1 - 1.5*IQR) & (df['price'] <= Q3 + 1.5*IQR)]
#Extreme price and mileage values were treated as outliers and removed using the IQR method to improve model robustness.

Data Pre-Processing

In [15]:
X = df.drop("price", axis=1)
y = df["price"]

In [21]:
#in modeling  models cannot be trained with categorical data , so we have to convert them into numbercal data if any available
#for that we use one hot encoding

X = pd.get_dummies(X, drop_first=True)

In [23]:
X.head()

,year,mileage,tax,mpg,engineSize,model_ 2 Series,model_ 3 Series,model_ 4 Series,model_ 5 Series,model_ 6 Series,...,model_ Z3,model_ Z4,model_ i3,model_ i8,transmission_Manual,transmission_Semi-Auto,fuelType_Electric,fuelType_Hybrid,fuelType_Other,fuelType_Petrol
0,2014,67068.0,125.0,57.6,2.0,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,2018,14827.0,145.0,42.8,2.0,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
2,2016,62794.0,160.0,51.4,3.0,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,2017,26676.0,145.0,72.4,1.5,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2014,39554.0,160.0,50.4,3.0,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [24]:
#scaling
#Feature scaling was applied to ensure all variables contribute equally to the model

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Train test split

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test , y_train , y_test = train_test_split(X_scaled,y, test_size=0.2, random_state=42)


Fitting models

In [29]:
#Linear Regression(Baseline)
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train,y_train)


LinearRegression()

In [30]:
#Random Forest Regressor

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [31]:
#XGBoost
from xgboost import XGBRegressor

xgb = XGBRegressor(random_state=42)
xgb.fit(X_train, y_train)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

Model Evaluation

In [33]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def evaluate(model):
    y_pred = model.predict(X_test)
    return {
        "MAE": mean_absolute_error(y_test, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "R2": r2_score(y_test, y_pred)
    }


In [34]:
evaluate(lr)

{'MAE': 2566.603518761905,
 'RMSE': np.float64(3381.1795194397787),
 'R2': 0.827827990231691}

In [35]:
evaluate(rf)

{'MAE': 1372.0726902190486,
 'RMSE': np.float64(1986.0007476636074),
 'R2': 0.9406002106344735}

In [36]:
evaluate(xgb)

{'MAE': 1344.7003548031762,
 'RMSE': np.float64(1928.2431848933118),
 'R2': 0.9440049418733483}

1. MAE (Mean Absolute Error)

Average error in price units

Lower = better

2. RMSE (Root Mean Squared Error)

Penalizes large mistakes

Lower = better

3. R² (R-squared)

How well the model explains the data

Closer to 1 = better